In [3]:
import os

os.chdir('../')

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [5]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [4]:
from Recommenders.Neural.Convolution.GraphConvolution import GraphConvolutionWithBlockMatrix, GraphConvolution
import optuna

def objective_function(trial):
    factors = trial.suggest_int("factors", 10, 500)
    alpha = trial.suggest_float("alpha", 0, 15)
    recommender = GraphConvolution(URM_train_validation)
    recommender.fit(num_factors=factors, alpha=alpha, random_seed=42)
    
    result_dic, _ = evaluator_test.evaluateRecommender(recommender)
    MAP = result_dic.loc[10]["MAP"]
    return MAP

class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/GraphConvolution/gf_firstTrial.csv", index = False)

optuna_study = optuna.create_study(study_name="GraphConvolution", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100,
                      )

[I 2024-12-30 17:21:59,170] A new study created in memory with name: GraphConvolution


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (498, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 3.15 min. Users per second: 188


[I 2024-12-30 17:32:45,073] Trial 0 finished with value: 0.034686656766933346 and parameters: {'factors': 498, 'alpha': 3.7139085869566326}. Best is trial 0 with value: 0.034686656766933346.


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (91, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 3.82 min. Users per second: 155


[I 2024-12-30 17:44:33,645] Trial 1 finished with value: 0.020240066265771933 and parameters: {'factors': 91, 'alpha': 6.271988717134312}. Best is trial 0 with value: 0.034686656766933346.


...

GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (255, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 3.69 min. Users per second: 161


[I 2024-12-30 18:18:52,796] Trial 4 finished with value: 0.02862001864003479 and parameters: {'factors': 255, 'alpha': 14.37888527078544}. Best is trial 0 with value: 0.034686656766933346.


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (100, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V


[W 2024-12-30 18:19:17,345] Trial 5 failed with parameters: {'factors': 100, 'alpha': 8.782889106263408} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_35677/262824940.py", line 8, in objective_function
    recommender.fit(num_factors=factors, alpha=alpha, random_seed=42)
  File "/Users/lucapagano/Developer/RecSys/Competizione/Recommenders/Neural/Convolution/GraphConvolution.py", line 54, in fit
    VTDotV = self.V.T.dot(self.V)
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/scipy/sparse/_compressed.py", line 85, in __init__
    self._coo_container(arg1, dtype=dtype)
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/scipy/sparse/_coo.py", line 190, in __init

KeyboardInterrupt: 

# We can already see that we need bigger factors

In [ ]:
from Recommenders.Neural.Convolution.GraphConvolution import GraphConvolution
import optuna

def objective_function(trial):
    factors = trial.suggest_int("factors", 500, 2500)
    alpha = trial.suggest_float("alpha", 0, 15)
    recommender = GraphConvolution(URM_train_validation)
    recommender.fit(num_factors=factors, alpha=alpha, random_seed=42)
    
    result_dic, _ = evaluator_test.evaluateRecommender(recommender)
    MAP = result_dic.loc[10]["MAP"]
    return MAP

class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/GraphConvolution/gf_secondTrial.csv", index = False)

optuna_study = optuna.create_study(study_name="GraphConvolution", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100,
                      )

[I 2024-12-30 18:40:58,341] A new study created in memory with name: GraphConvolution


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (1608, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 3.28 min. Users per second: 181


[I 2024-12-30 18:52:50,249] Trial 0 finished with value: 0.043327983241803945 and parameters: {'factors': 1608, 'alpha': 5.051769446290934}. Best is trial 0 with value: 0.043327983241803945.


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (2236, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 4.44 min. Users per second: 134


[I 2024-12-30 19:06:24,287] Trial 1 finished with value: 0.0454054553319184 and parameters: {'factors': 2236, 'alpha': 6.009407100047718}. Best is trial 1 with value: 0.0454054553319184.


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (2381, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 3.41 min. Users per second: 174


[I 2024-12-30 19:18:45,254] Trial 2 finished with value: 0.045802090092016766 and parameters: {'factors': 2381, 'alpha': 13.65337908117169}. Best is trial 2 with value: 0.045802090092016766.


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (1960, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 2.75 min. Users per second: 216


[I 2024-12-30 19:30:12,873] Trial 3 finished with value: 0.044659397944689344 and parameters: {'factors': 1960, 'alpha': 13.232627727638812}. Best is trial 2 with value: 0.045802090092016766.


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (2302, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I


# Even higher

In [4]:
from Recommenders.Neural.Convolution.GraphConvolution import GraphConvolution
import optuna

def objective_function(trial):
    factors = trial.suggest_int("factors", 1000, 3500)
    alpha = trial.suggest_float("alpha", 0, 15)
    recommender = GraphConvolution(URM_train_validation)
    recommender.fit(num_factors=factors, alpha=alpha, random_seed=42)
    
    result_dic, _ = evaluator_test.evaluateRecommender(recommender)
    MAP = result_dic.loc[10]["MAP"]
    return MAP

class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/GraphConvolution/gf_thirdTrial.csv", index = False)

optuna_study = optuna.create_study(study_name="GraphConvolution", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100,
                      )

[I 2024-12-30 19:40:20,841] A new study created in memory with name: GraphConvolution


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (2341, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 2.83 min. Users per second: 210


[I 2024-12-30 19:52:35,151] Trial 0 finished with value: 0.04620460894944539 and parameters: {'factors': 2341, 'alpha': 0.7174368134138076}. Best is trial 0 with value: 0.04620460894944539.


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (3033, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 2.75 min. Users per second: 215


[I 2024-12-30 20:05:09,533] Trial 1 finished with value: 0.04680404885651251 and parameters: {'factors': 3033, 'alpha': 7.23175879418644}. Best is trial 1 with value: 0.04680404885651251.


...

GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (1427, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 2.82 min. Users per second: 211


[I 2024-12-30 21:03:10,297] Trial 6 finished with value: 0.04309368178605299 and parameters: {'factors': 1427, 'alpha': 1.2999532721318836}. Best is trial 1 with value: 0.04680404885651251.


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (1739, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V


[W 2024-12-30 21:09:28,356] Trial 7 failed with parameters: {'factors': 1739, 'alpha': 2.669393926880328} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_65980/2032058779.py", line 8, in objective_function
    recommender.fit(num_factors=factors, alpha=alpha, random_seed=42)
  File "/Users/lucapagano/Developer/RecSys/Competizione/Recommenders/Neural/Convolution/GraphConvolution.py", line 58, in fit
    partial_res = self.alpha * self.D_I_inv.dot(VTDotV)
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/scipy/sparse/_base.py", line 620, in __rmul__
    return self._rmul_dispatch(other)
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/scipy/sparse/_base.py", line

KeyboardInterrupt: 

# Ok we need more factors and run a longer trial all night

In [ ]:
from Recommenders.Neural.Convolution.GraphConvolution import GraphConvolution
import optuna

def objective_function(trial):
    factors = trial.suggest_int("factors", 2500, 7000)
    alpha = trial.suggest_float("alpha", 0, 15)
    recommender = GraphConvolution(URM_train_validation)
    recommender.fit(num_factors=factors, alpha=alpha, random_seed=42)
    
    result_dic, _ = evaluator_test.evaluateRecommender(recommender)
    MAP = result_dic.loc[10]["MAP"]
    return MAP

class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/GraphConvolution/gf_fourthTrial.csv", index = False)

optuna_study = optuna.create_study(study_name="GraphConvolution", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100,
                      )

[I 2024-12-30 21:10:15,290] A new study created in memory with name: GraphConvolution


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (2760, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 2.84 min. Users per second: 209


[I 2024-12-30 21:22:29,845] Trial 0 finished with value: 0.046335783739911714 and parameters: {'factors': 2760, 'alpha': 11.901829407351755}. Best is trial 0 with value: 0.046335783739911714.


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (2648, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 2.61 min. Users per second: 228


[I 2024-12-30 21:34:12,688] Trial 1 finished with value: 0.04633034000227268 and parameters: {'factors': 2648, 'alpha': 6.461344645519348}. Best is trial 0 with value: 0.046335783739911714.


...

GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (5678, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 2.65 min. Users per second: 224


[I 2024-12-31 09:28:34,083] Trial 54 finished with value: 0.047944824603061244 and parameters: {'factors': 5678, 'alpha': 2.625856889580654}. Best is trial 12 with value: 0.04890634385514104.


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (6001, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
EvaluatorHoldout: Processed 35595 (100.0%) in 2.12 min. Users per second: 279


[I 2024-12-31 09:41:34,362] Trial 55 finished with value: 0.048061401748505 and parameters: {'factors': 6001, 'alpha': 1.2393675283284893}. Best is trial 12 with value: 0.04890634385514104.


GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (5225, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V


# We can stop here, we can start the submission
I must test if having low factors despite a worse validation is less overfitted and more beneficial to the hybrid!  

Top 10 is 


| Rank | Result       | Factors | Alpha     |
|------|--------------|---------|-----------|
| 1    | 0.048906     | 5645.0  | 0.223329  |
| 2    | 0.048898     | 6103.0  | 0.203742  |
| 3    | 0.048887     | 5892.0  | 0.252335  |
| 4    | 0.048846     | 6519.0  | 0.140354  |
| 5    | 0.048833     | 5713.0  | 0.182465  |
| 6    | 0.048809     | 5705.0  | 0.335314  |
| 7    | 0.048588     | 5645.0  | 0.126542  |
| 8    | 0.048569     | 6505.0  | 0.488692  |
| 9    | 0.048517     | 5677.0  | 0.815955  |
| 10   | 0.048489     | 6040.0  | 0.671239  |
| 11   | 0.048449     | 5408.0  | 0.780694  |
| 12   | 0.048435     | 5503.0  | 0.840568  |
| 13   | 0.048427     | 5553.0  | 0.892189  |
| 14   | 0.048365     | 6141.0  | 0.857391  |
| 15   | 0.048284     | 4753.0  | 0.112754  |
| 16   | 0.048258     | 3826.0  | 0.286909  |
| 17   | 0.048231     | 5827.0  | 0.081986  |
| 18   | 0.048185     | 5387.0  | 1.165494  |
| 19   | 0.048138     | 4441.0  | 1.320820  |
| 20   | 0.048102     | 5885.0  | 1.331775  |



We can see that entry 16 despite having a sensible lower amount of num factors had a similar MAP. 

Let's first do a submission with the best parameters and then one with entry 16

In [13]:
from Recommenders.Neural.Convolution.GraphConvolution import GraphConvolution

best_params = {'num_factors' : 5645, 'alpha' : 0.223329, 'random_seed':42}

recommender = GraphConvolution(URM_all)
recommender.fit(**best_params)

target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
num_factors = best_params["num_factors"]
alpha = best_params["alpha"]
submission.to_csv(f"submissions/graphConvolution_num_factors:{num_factors}_alpha:_{alpha}.csv", index=False)

GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (5645, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9


In [14]:
W_sparse = recommender.W_sparse

sps.save_npz(f'graphConvolution_W_sparse_num_factors:{num_factors}_alpha:_{alpha}.npz', W_sparse)

# Let's also save the W_sparse with only train and validation for hybrid

In [15]:
import gc
gc.collect()

0

In [16]:
from Recommenders.Neural.Convolution.GraphConvolution import GraphConvolution

best_params = {'num_factors' : 5645, 'alpha' : 0.223329, 'random_seed':42}

recommender = GraphConvolution(URM_train_validation)
recommender.fit(**best_params)

W_sparse = recommender.W_sparse

sps.save_npz(f'graphConvolution_W_sparse_TRAINVAL_num_factors:{num_factors}_alpha:_{alpha}.npz', W_sparse)

GraphConvolution: Computing SVD decomposition of the normalized adjacency matrix...
Computing first term
D_I_inv shape: (38121, 38121)
V shape: (5645, 38121)
D_I shape: (38121, 38121)
Computing second term: V^T * V
Computing third term: alpha * D_I_inv * V^T * V
Computing fourth term: third_term * D_I
Computed Similarity matrix
